#1. Ejemplo del método de clasificación Naive Bayes
#Crop mapping using fused optical-radar data set Data Set

Este gran conjunto de datos es un radar óptico bitemporal fusionado para la clasificación de tierras de cultivo. Las imágenes fueron recolectadas por satélites RapidEye (ópticos) y el sistema de radar de apertura sintética de vehículos aéreos no tripulados (UAVSAR) (Radar) sobre una región agrícola cerca de Winnipeg, Manitoba, Canadá en 2012.
Hay 2 * 49 funciones de radar y 2 * 38 funciones ópticas para dos fechas: 05 y 14 de julio de 2012.

Existen siete clases de tipos de cultivos para este conjunto de datos de la siguiente manera: 1-Maíz; 2-guisantes; 3- Canola; 4-soja; 5- Avena; 6- Trigo; y 7 de hoja ancha.

Más: https://archive.ics.uci.edu/ml/datasets/Crop+mapping+using+fused+optical-radar+data+set





#2. Importación de librerías y montando google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import ____ as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
import ____ as np
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as ____
import cv2
import random
import ____ as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier


In [ ]:
path = r'/content/drive/Shareddrives/Data Science para Geociencias/7. Modelos de ensamble/7.2 Bagging'
train_path = os.path.join(path,'WinnipegDataset.txt')

# 3. Lectura y visualuzación del dataset

In [ ]:
train_df = pd.read_csv(____)
clases = train_df['label'].unique()
train_df.dropna(axis=0, inplace=True)
____.head(10)

#3. Escalamiento y codificación de etiquetas

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_df.loc[:, train_df.columns != 'label'] = scaler.fit_transform(____.loc[:, train_df.columns != 'label'])

In [ ]:
le = LabelEncoder()
train_df['label'] = le.fit_transform(____.label.values)
_____.head(10)

#4.  Selección de características

In [ ]:
fs = SelectKBest(score_func=f_classif, k=25)
np_X = _____.iloc[:, 1:].to_numpy()
np_Y = train_df.iloc[:, 0].to_numpy()
X_selec = fs.fit_transform(np_X, np_Y)
attr_names = train_df.columns.values.tolist()
# Lista de bools que indican si es un atributo seleccionado o no
mask = fs.get_support()
# Lista de los K mejores atributos
new_features = [] 
for bool, feature in zip(mask, attr_names):
    if bool:
        new_features.append(feature)
new_train_df = pd.DataFrame(X_selec, columns=new_features)
new_train_df['label'] = train_df['label']

In [ ]:
_____.head(10)

#5. Dividiendo el conjunto en entrenamiento y prueba

In [ ]:
X = np.array(new_train_df.iloc[:,1:])
y = np.array(new_train_df['label'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.75, random_state = 0)

print(f'Dimension de los datos de entrenamiento {____.shape}')
print(f'Dimension de los datos de prueba {____.shape}')

#6. Entrenando y prediciendo con CART

In [ ]:
seed = 10
n_classes = 7
dectree = DecisionTreeClassifier(max_depth= 50, random_state=____)
dectree = dectree.fit(X_train,y_train)

In [ ]:
y_pred = dectree.predict(X_test)

In [ ]:
score = metrics.accuracy_score(y_test, y_pred)
print("Test Acc: %s"%score)

Test Acc: 0.9722026712934168


In [ ]:
#Funciones para MC y métricas
def plot_confusion_matrix(cm, classes, tit, normalize=False):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)
        title, fmt = 'Matriz de confusión normalizada', '.2f'
    else:
        title, fmt = tit, 'd'
    plt.figure(figsize=(10,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)#, fontsize=12)
    plt.colorbar(pad=0.05)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=40)
    plt.yticks(tick_marks, classes)
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center", 
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Clase Verdadera')#, fontsize=10)
    plt.xlabel('Clase Predicha')#, fontsize=10)
    plt.savefig(title+'.png')
    #plt.grid(b=None)
    plt.show()
def sens_spec(cls_names, y_true, y_pred, pesos):
  sensitivity = []
  specificity = []
  prec=[]
  fscore = []
  sd = []
  for i,name in enumerate(cls_names):
    TP = np.sum((y_true==name) & (y_pred==name))
    TN = np.sum((y_true!=name) & (y_pred!=name))
    FP = np.sum((y_true!=name) & (y_pred==name))
    FN = np.sum((y_true==name) & (y_pred!=name))
    sensitivity.append(TP/(TP+FN))
    specificity.append(FP/(TN+FP))
    prec.append(TP/(TP+FP))
    fscore.append(2*(prec[i]*sensitivity[i])/(prec[i]+sensitivity[i]))
  sensitivity.append(sum([x*y for x,y in zip(sensitivity,pesos)]))
  specificity.append(sum([x*y for x,y in zip(specificity,pesos)]))
  prec.append(sum([x*y for x,y in zip(prec,pesos)]))
  fscore.append(sum([x*y for x,y in zip(fscore,pesos)]))
  for i in range(len(cls_names)):
    sd.append(np.sqrt((sensitivity[i]-sensitivity[-1])**2))
  sd.append(np.nan)
  return sensitivity, specificity, prec, fscore, sd

In [ ]:
predictions = np.float32(____)
true_labels = np.float32(____)
cnf_matrix = confusion_matrix(true_labels, predictions, labels=range(n_classes))
tit = 'Matriz de confusión árbol de decisión (CART)'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=True)

In [ ]:
clases = train_df['label'].unique().tolist()
n_clases = len(clases)
pesos = train_df['label'].value_counts().sort_index().tolist()/ np.sum(train_df['label'].value_counts().tolist())
sensitivity, FPR, prec, fscore, sd = sens_spec(range(n_clases), true_labels, predictions, pesos)
d = {'TPR':sensitivity, 'FPR':FPR, 'Acc':prec, 'F-score':fscore}
ind = clases+['Promedio']
df = pd.DataFrame(d, index=ind)#.transpose()
index = df.index
precprom = sum([x*y for x,y in zip(prec,pesos)])
index.name = 'Acc Pond: %s'%precprom
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df, annot=True)

#7. Entrenando y prediciendo con Bagging

In [ ]:
cart =  DecisionTreeClassifier(random_state=seed)
bagging = BaggingClassifier(base_estimator=cart,n_estimators=20, random_state=0).fit(X_train, y_train)
y_pred = bagging.predict(X_test)

In [ ]:
predictions = np.float32(y_pred)
true_labels = np.float32(y_test)
cnf_matrix = confusion_matrix(true_labels, predictions, labels=range(n_classes))
tit = 'Matriz de confusión AdaBoost'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=True)

In [ ]:
sensitivity, FPR, prec, fscore, sd = sens_spec(range(n_clases), true_labels, predictions, pesos)
d = {'TPR':sensitivity, 'FPR':FPR, 'Acc':prec, 'F-score':fscore}
ind = clases+['Promedio']
df = pd.DataFrame(d, index=ind)#.transpose()
index = df.index
precprom = sum([x*y for x,y in zip(prec,pesos)])
index.name = 'Acc Pond: %s'%precprom
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df, annot=True)